<a href="https://colab.research.google.com/github/Gaurab-B/MRS/blob/main/Week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = "/content/drive/My Drive/Colab Notebooks/week6/movies.csv"                                     #MOVIES.CSV
moviescsv = pd.read_csv(path, delimiter=",",header = None,names=["movieId", "Title", "Genres"])
print(moviescsv.head(5))

   movieId                     Title  \
0  movieId                     title   
1        1          Toy Story (1995)   
2        2            Jumanji (1995)   
3        3   Grumpier Old Men (1995)   
4        4  Waiting to Exhale (1995)   

                                        Genres  
0                                       genres  
1  Adventure|Animation|Children|Comedy|Fantasy  
2                   Adventure|Children|Fantasy  
3                               Comedy|Romance  
4                         Comedy|Drama|Romance  


In [3]:
moviescsv.drop_duplicates(subset="Title", keep='last',inplace=True)  #The 'inplace=True' ensures that the original 'movies' DataFrame is modified.
moviescsv

,movieId,Title,Genres
0,movieId,title,genres
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,3,Grumpier Old Men (1995),Comedy|Romance
4,4,Waiting to Exhale (1995),Comedy|Drama|Romance
...,...,...,...
62419,209157,We (2018),Drama
62420,209159,Window of the Soul (2001),Documentary
62421,209163,Bad Poems (2018),Comedy|Drama
62422,209169,A Girl Thing (2001),(no genres listed)


In [4]:
import re #regularexpressions
import string
def clean_text_func(text):
    '''Make text lowercase, removing special characters'''
    text = text.lower()
    # remove multiple whitespaces
    text = re.sub('\s+',' ', text)
    # remove characters that not words
    text=re.sub('[^a-zA-Z0-9]',' ',text)
    return text

clean_text = lambda x: clean_text_func(x)    #This lambda function allows us to apply the cleaning function conveniently in a single line.

In [5]:
for col in moviescsv.columns:
    moviescsv[col+'_cleaned'] = moviescsv[col].apply(clean_text)

In [6]:
moviescsv.head()

,movieId,Title,Genres,movieId_cleaned,Title_cleaned,Genres_cleaned
0,movieId,title,genres,movieid,title,genres
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,toy story 1995,adventure animation children comedy fantasy
2,2,Jumanji (1995),Adventure|Children|Fantasy,2,jumanji 1995,adventure children fantasy
3,3,Grumpier Old Men (1995),Comedy|Romance,3,grumpier old men 1995,comedy romance
4,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4,waiting to exhale 1995,comedy drama romance


In [7]:
moviescsv.isnull().sum()

movieId            0
Title              0
Genres             0
movieId_cleaned    0
Title_cleaned      0
Genres_cleaned     0
dtype: int64

In [8]:
if 'all_data' not in moviescsv:                #this lines prevent from re-runing the data which causes multiple datasets.
    moviescsv['all_data'] = moviescsv[moviescsv.columns[3:]].apply(
        lambda x: ' '.join(x.astype(str)),
        axis=1
        )

In [9]:
moviescsv.columns

Index(['movieId', 'Title', 'Genres', 'movieId_cleaned', 'Title_cleaned',
       'Genres_cleaned', 'all_data'],
      dtype='object')

In [10]:
moviescsv.head()

,movieId,Title,Genres,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
0,movieId,title,genres,movieid,title,genres,movieid title genres
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,toy story 1995,adventure animation children comedy fantasy,1 toy story 1995 adventure animation childre...
2,2,Jumanji (1995),Adventure|Children|Fantasy,2,jumanji 1995,adventure children fantasy,2 jumanji 1995 adventure children fantasy
3,3,Grumpier Old Men (1995),Comedy|Romance,3,grumpier old men 1995,comedy romance,3 grumpier old men 1995 comedy romance
4,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4,waiting to exhale 1995,comedy drama romance,4 waiting to exhale 1995 comedy drama romance


In [11]:
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import re
nltk.download('wordnet')

def lemma_tokens(tokens, lemma):
    lemma_result = [lemma.lemmatize(item) for item in tokens]
    return(lemma_result)

def tokenize(text):
    lemma = nltk.WordNetLemmatizer()
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = nltk.word_tokenize(text)

    # English Stop words
    # Re-add the additional stop words since we are recreating the document-term matrix
    stop_words = stopwords.words('english')

    # Remove all stop words: no_stops
    no_stops = [t for t in tokens if t not in stop_words]

    lemma = lemma_tokens(no_stops, lemma)
    return(lemma)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

# Re-add the additional stop words since we are recreating the document-term matrix
stop_words = stopwords.words('english')

# stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

vect = TfidfVectorizer(stop_words= stop_words,
                       analyzer='word',
                       max_features = 60000,
                       tokenizer = tokenize,
                       lowercase = True,
                       )

# Fit and transform
nltk.download('punkt')              #this needs to be added or else the code wont run.
nltk.download('wordnet')
X = vect.fit_transform(moviescsv.all_data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
X.shape

(62326, 31061)

In [14]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
feature_names = vect.get_feature_names_out()
df_tfid = pd.DataFrame(X.toarray(), columns=feature_names)
df_tfid

,aaaaaaaah,aachi,aadhavan,aadhi,aadu,aadukalam,aag,aaicha,aaj,aaja,...,zwerge,zwierze,zwijgen,zwischen,zwischenf,zycia,zycie,zyciu,zygote,zyzzyx
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
import re                                                                           #THIS REMOVES ANY SUMBOLS

def extract_text(input_string):
    text_only = re.sub(r'[^a-zA-Z0-9\s]', '', input_string)
    return text_only.strip()

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
def querying_movie(query, K):
    cleaned_query = query[0].strip().lower()
    cleaned_query = extract_text(cleaned_query)
    unique_movie_titles = pd.Series(moviescsv['Title_cleaned']).str.strip().unique()
    if len(cleaned_query) < 3:
        return 'Error! The query must contain at least 3 characters.'
    matching_titles = unique_movie_titles[pd.Series(unique_movie_titles).str.contains(cleaned_query)]
    if len(matching_titles) == 0:
        return 'Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies'
    else:
        matching_indices = [list(unique_movie_titles).index(title) for title in matching_titles]
        query_vectors = X[matching_indices]

        similarity_scores = cosine_similarity(query_vectors, X)
        top_indices = similarity_scores.argsort()[0][::-1][1:K+1]   #//gives index of similiar ones, 0 is the first row that stores index
        recommended_movies = unique_movie_titles[top_indices]
        return list(recommended_movies)


In [18]:
# test case 1
K = 10 #-- list first K recommended movie names
query = ['batman']
querying_movie(query, K)

['batman  1966',
 'batman  1989',
 'batman  1943',
 'batman returns  1992',
 'slightly scarlet  1956',
 'la scelta  2015',
 'batman and robin  1949',
 'forever  2006',
 'forget me not  2010',
 'now and forever  1934']

In [20]:
# test case 1
K = 10 #-- list first K recommended movie names
query = ['avatar']
querying_movie(query, K)

['my avatar and me  min avatar og mig   2010',
 'after earth  2013',
 'more  1998',
 'godzilla  2014',
 'avengers  the  2012',
 'oblivion  2013',
 'spider man 2  2004',
 'iron man 2  2010',
 'spider man 3  2007',
 'i am legend  2007']

In [21]:
path2 = "/content/drive/My Drive/Colab Notebooks/week6/ratings.csv"                                 #RATINGS.CSV
ratings =pd.read_csv(path2, delimiter=",", header=None, names=["userId", "movieId", "Rating", "Timestamp"])

#Print the first 5 rows of the DataFrame

print(ratings.head(5))

<ipython-input-21-b07c5fd17294>:2: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings =pd.read_csv(path2, delimiter=",", header=None, names=["userId", "movieId", "Rating", "Timestamp"])


   userId  movieId  Rating   Timestamp
0  userId  movieId  rating   timestamp
1       1      296     5.0  1147880044
2       1      306     3.5  1147868817
3       1      307     5.0  1147868828
4       1      665     5.0  1147878820


In [22]:
movie_id = 4

def Amovies(movie_id):
  movie = moviescsv.loc[movie_id]
  return movie

Amovies(movie_id)

movieId                                                          4
Title                                     Waiting to Exhale (1995)
Genres                                        Comedy|Drama|Romance
movieId_cleaned                                                  4
Title_cleaned                             waiting to exhale  1995 
Genres_cleaned                                comedy drama romance
all_data           4 waiting to exhale  1995  comedy drama romance
Name: 4, dtype: object

In [23]:
ratings.dtypes

userId       object
movieId      object
Rating       object
Timestamp    object
dtype: object

In [24]:
moviescsv = moviescsv.drop(moviescsv.index[0])
ratings = ratings.drop(ratings.index[0])

In [25]:
ratings

,userId,movieId,Rating,Timestamp
1,1,296,5.0,1147880044
2,1,306,3.5,1147868817
3,1,307,5.0,1147868828
4,1,665,5.0,1147878820
5,1,899,3.5,1147868510
...,...,...,...,...
25000091,162541,50872,4.5,1240953372
25000092,162541,55768,2.5,1240951998
25000093,162541,56176,2.0,1240950697
25000094,162541,58559,4.0,1240953434


In [26]:
ratings.Rating = pd.to_numeric(ratings.Rating )

In [27]:
similar_users = ratings[(ratings.movieId == movie_id) & (ratings.Rating > 4)]
similar_users

,userId,movieId,Rating,Timestamp
409677,2827,4,4.5,1259446061
651489,4458,4,5.0,835458536
667835,4568,4,5.0,846471991
783112,5298,4,5.0,1041809702
791972,5350,4,5.0,834249392
...,...,...,...,...
24669931,160298,4,5.0,830879187
24719372,160662,4,5.0,861403227
24763398,160944,4,5.0,854018150
24808549,161245,4,5.0,844889201


In [28]:
similar_users = ratings[(ratings.movieId == movie_id) & (ratings.Rating > 4)]["userId"].unique()
  #this finds similar users who also watched the same movie and rated at least 5 on the movie. who have similar taste to us.
similar_users

array([2827, 4458, 4568, 5298, 5350, 6039, 6518, 7888, 8104, 9554, 9643,
       11038, 11924, 11930, 12810, 13917, 15029, 15328, 15406, 15565,
       16041, 16570, 17220, 18615, 18619, 19915, 20291, 20537, 22456,
       23121, 23632, 24935, 27085, 27987, 29475, 29847, 30488, 30645,
       30692, 32896, 33525, 34000, 34574, 34630, 36432, 37055, 37841,
       39863, 40126, 40293, 40439, 40692, 41562, 42632, 42914, 43060,
       45511, 46859, 48917, 48957, 48991, 52279, 53666, 53802, 54111,
       59102, 59585, 59737, 60540, 60876, 61361, 61428, 61452, 63442,
       63787, 63900, 64628, 65370, 66416, 66672, 67519, 68953, 69475,
       69733, 71136, 71724, 72863, 74282, 75209, 75498, 75697, 77179,
       77897, 77994, 78472, 80817, 81063, 81750, 82383, 82542, 82717,
       82897, 83605, 84072, 84981, 85968, 86271, 86728, 88165, 90411,
       91996, 92155, 92891, 94206, 94556, 97855, 98123, 98961, 100118,
       100657, 102118, 103817, 105076, 106363, 107467, 110827, 114114,
       115040, 

 This line filters the "ratings" dataset based on two conditions. It selects rows where the "movieId" column matches the specified "movie_id" value and the "rating" column is greater than 4. It then extracts the unique values from the "userId" column for these filtered rows. The resulting array contains the user IDs of users who have rated the specified movie highly (rating > 4).

In [29]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings.Rating >4)]    #finding similar movie.
similar_user_recs


,userId,movieId,Rating,Timestamp
409677,2827,4,4.5,1259446061
409700,2827,203,5.0,1259446950
409706,2827,253,4.5,1262902988
409709,2827,266,5.0,1261972912
409710,2827,288,5.0,1259447768
...,...,...,...,...
24814520,161296,36,5.0,844824223
24814523,161296,52,5.0,844824332
24814533,161296,194,5.0,844824289
24814535,161296,608,5.0,844824243


In [30]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings.Rating >4)]["movieId"]
similar_user_recs

409677        4
409700      203
409706      253
409709      266
409710      288
           ... 
24814520     36
24814523     52
24814533    194
24814535    608
24814542    778
Name: movieId, Length: 13363, dtype: object

This line filters the "ratings" dataset based on two conditions. It selects rows where the "userId" column matches any of the user IDs in the "similar_users" array and the "rating" column is greater than 4. It then extracts the "movieId" column for these filtered rows. The resulting array contains the movie IDs for movies that have been highly rated by the similar users identified in the previous step.

In [31]:
similar_user_recs.value_counts()

4         182
356        85
364        77
318        73
508        67
         ... 
101864      1
100390      1
98203       1
96490       1
1410        1
Name: movieId, Length: 3393, dtype: int64

In [32]:
similar_user_recs.value_counts() / len(similar_users)

4         1.000000
356       0.467033
364       0.423077
318       0.401099
508       0.368132
            ...   
101864    0.005495
100390    0.005495
98203     0.005495
96490     0.005495
1410      0.005495
Name: movieId, Length: 3393, dtype: float64

In [33]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)               #just keep 10% of all the movies .
similar_user_recs = similar_user_recs[similar_user_recs > .10]
similar_user_recs

4       1.000000
356     0.467033
364     0.423077
318     0.401099
508     0.368132
          ...   
262     0.104396
1097    0.104396
1721    0.104396
1073    0.104396
858     0.104396
Name: movieId, Length: 126, dtype: float64

This line calculates the relative frequency of each movie ID in the "similar_user_recs" array. It counts the occurrences of each movie ID and divides it by the total number of similar users identified in step 1. The resulting values represent the proportion of similar users who have rated each movie highly. This line further filters the "similar_user_recs" series by selecting only those movie IDs that have a relative frequency greater than 0.10 (or 10%). This step helps narrow down the recommended movies to those that are highly rated by a significant portion of similar users.

Above I found the list of movies that have same interest to me.

Now I find how much all users in the dataset liked the movies.

In [34]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings.Rating > 4)]      #people who are in similar_users_list and people who have rated them highly.
all_users


,userId,movieId,Rating,Timestamp
131150,928,50,5.0,1553179275
131151,928,110,5.0,1553179282
131153,928,318,5.0,1553179010
131155,928,527,5.0,1553179022
131190,929,356,5.0,1166876239
...,...,...,...,...
24999917,162541,50,5.0,1240953428
24999924,162541,260,5.0,1240952836
24999930,162541,345,4.5,1240950041
24999935,162541,527,4.5,1240953464


 This line filters the "ratings" dataset based on two conditions. It selects rows where the "movieId" column matches any of the movie IDs in the index of the "similar_user_recs" series (i.e., the highly rated movies by similar users) and the "rating" column is greater than 4. The resulting dataset, "all_users," contains the userID of all users who have rated any of the recommended movies highly.

In [35]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())   #find the movies that all_user dataset who have watched movies recommend to us.
all_user_recs        #percentage of all users that liked the movied

318    0.374360
296    0.311447
356    0.257428
593    0.247004
260    0.243086
         ...   
46     0.002244
195    0.001413
4      0.001326
122    0.001151
271    0.000940
Name: movieId, Length: 126, dtype: float64

There is a difference between someone who watches movies like us and regular user.
If I watch titanic and people simlar to me watched toy story after that beacuse of their choice BUT all people in the dataset also watched toy story. Then toy story is not a good recommendation .

In [36]:
percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
percentages.columns = ["Users Like Us", "Average Person"]

In [37]:
percentages

,Users Like Us,Average Person
4,1.000000,0.001326
356,0.467033,0.257428
364,0.423077,0.093798
318,0.401099,0.374360
508,0.368132,0.037105
...,...,...
262,0.104396,0.007452
1097,0.104396,0.066868
1721,0.104396,0.056503
1073,0.104396,0.053291


In [38]:
percentages["score"] = percentages["Users Like Us"] / percentages["Average Person"]     #score is the ration between how much similar users to how much average person liked the movie.
percentages = percentages.sort_values("score", ascending=False)
percentages

,Users Like Us,Average Person,score
4,1.000000,0.001326,754.313187
122,0.142857,0.001151,124.127486
271,0.115385,0.000940,122.795170
195,0.131868,0.001413,93.317095
46,0.142857,0.002244,63.675788
...,...,...,...
50,0.181319,0.219201,0.827180
296,0.241758,0.311447,0.776242
608,0.115385,0.157781,0.731295
260,0.131868,0.243086,0.542476


In [39]:
moviescsv.movieId_cleaned = moviescsv.movieId_cleaned.replace(np.nan, 0);

In [40]:
print(moviescsv["movieId_cleaned"].unique())

['1' '2' '3' ... '209163' '209169' '209171']


In [41]:
type(percentages.index.dtype)

numpy.dtype[int64]

In [42]:
ratings.Rating = pd.to_numeric(ratings.Rating )

In [70]:
moviescsv["movieId_cleaned"] = moviescsv["movieId_cleaned"].astype(percentages.index.dtype)

merged_data = percentages.head(10).merge(moviescsv, left_index=True, right_on="movieId_cleaned")
merged_data

,Users Like Us,Average Person,score,movieId,Title,Genres,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
4,1.000000,0.001326,754.313187,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4,waiting to exhale 1995,comedy drama romance,4 waiting to exhale 1995 comedy drama romance
121,0.142857,0.001151,124.127486,122,Boomerang (1992),Comedy|Romance,122,boomerang 1992,comedy romance,122 boomerang 1992 comedy romance
269,0.115385,0.000940,122.795170,271,Losing Isaiah (1995),Drama,271,losing isaiah 1995,drama,271 losing isaiah 1995 drama
194,0.131868,0.001413,93.317095,195,Something to Talk About (1995),Comedy|Drama|Romance,195,something to talk about 1995,comedy drama romance,195 something to talk about 1995 comedy dram...
46,0.142857,0.002244,63.675788,46,How to Make an American Quilt (1995),Drama|Romance,46,how to make an american quilt 1995,drama romance,46 how to make an american quilt 1995 drama ...
473,0.247253,0.003926,62.976055,477,What's Love Got to Do with It? (1993),Drama|Musical,477,what s love got to do with it 1993,drama musical,477 what s love got to do with it 1993 dram...
377,0.214286,0.004261,50.287546,381,When a Man Loves a Woman (1994),Drama|Romance,381,when a man loves a woman 1994,drama romance,381 when a man loves a woman 1994 drama romance
217,0.159341,0.003212,49.603361,218,Boys on the Side (1995),Comedy|Drama,218,boys on the side 1995,comedy drama,218 boys on the side 1995 comedy drama
202,0.208791,0.004232,49.335458,203,"To Wong Foo, Thanks for Everything! Julie Newm...",Comedy,203,to wong foo thanks for everything julie newm...,comedy,203 to wong foo thanks for everything julie ...
347,0.104396,0.002418,43.168526,351,"Corrina, Corrina (1994)",Comedy|Drama|Romance,351,corrina corrina 1994,comedy drama romance,351 corrina corrina 1994 comedy drama romance


In [71]:
def find_similar_movies(movie_id):
  ratings.Rating = pd.to_numeric(ratings.Rating )
  similar_users = ratings[(ratings.movieId == movie_id) & (ratings.Rating > 4)]["userId"].unique()
  similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings.Rating >4)]["movieId"]
  similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
  similar_user_recs = similar_user_recs[similar_user_recs > .10]

  all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings.Rating > 4)]
  all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
  percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
  percentages.columns = ["Users Like Us", "Average Person"]
  percentages["score"] = percentages["Users Like Us"] / percentages["Average Person"]
  percentages = percentages.sort_values("score", ascending=False)
  moviescsv["movieId_cleaned"] = moviescsv["movieId_cleaned"].astype(percentages.index.dtype)
  merged_data = percentages.head(10).merge(moviescsv, left_index=True, right_on="movieId_cleaned")
  df = merged_data.drop(['movieId', 'Genres'], axis=1)
  return df

In [72]:
find_similar_movies(2)

,Users Like Us,Average Person,score,Title,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
2,1.000000,0.017544,57.000000,Jumanji (1995),2,jumanji 1995,adventure children fantasy,2 jumanji 1995 adventure children fantasy
157,0.104827,0.005603,18.708383,Casper (1995),158,casper 1995,adventure children,158 casper 1995 adventure children
314,0.140271,0.009420,14.890449,"Santa Clause, The (1994)",317,santa clause the 1994,comedy drama fantasy,317 santa clause the 1994 comedy drama fantasy
579,0.190799,0.020435,9.337002,Home Alone (1990),586,home alone 1990,children comedy,586 home alone 1990 children comedy
496,0.265460,0.030503,8.702668,Mrs. Doubtfire (1993),500,mrs doubtfire 1993,comedy drama,500 mrs doubtfire 1993 comedy drama
363,0.194193,0.022380,8.677210,"Mask, The (1994)",367,mask the 1994,action comedy crime fantasy,367 mask the 1994 action comedy crime fantasy
2527,0.119910,0.015103,7.939553,"Mummy, The (1999)",2617,mummy the 1999,action adventure comedy fantasy horror thriller,2617 mummy the 1999 action adventure comedy...
722,0.164027,0.021811,7.520473,Twister (1996),736,twister 1996,action adventure romance thriller,736 twister 1996 action adventure romance th...
580,0.190045,0.029915,6.352941,Ghost (1990),587,ghost 1990,comedy drama fantasy romance thriller,587 ghost 1990 comedy drama fantasy romance ...
313,0.171192,0.027427,6.241679,Stargate (1994),316,stargate 1994,action adventure sci fi,316 stargate 1994 action adventure sci fi


In [73]:
import re

In [74]:
import re #regularexpressions
import string
def clean_text_func(text):
    '''Make text lowercase, removing special characters'''
    text = text.lower()
    # remove multiple whitespaces
    text = re.sub('\s+',' ', text)
    # remove characters that not words
    text=re.sub('[^a-zA-Z0-9]',' ',text)
    return text

clean_text = lambda x: clean_text_func(x)    #This lambda function allows us to apply the cleaning function conveniently in a single line.

In [184]:
def findmovie(name):
  cleaned_query = clean_text_func(name);
  cleaned_query = cleaned_query.replace("  ", " ").strip()
  if len(cleaned_query) < 3:
    return 'Error! The query must contain at least 3 characters.'
  unique_movie_titles = pd.Series(moviescsv['all_data']).str.strip().unique()
  matching_titles = [title for title in unique_movie_titles if all(word.lower() in title.lower() for word in cleaned_query.split())]
  if len(matching_titles) == 0:
    return 'Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies'
  else:
    matching_ids = [moviescsv[moviescsv['all_data'].str.strip() == title]['movieId_cleaned'].iloc[0] for title in matching_titles]
    last_movie_id = matching_ids[0]
    return find_similar_movies(last_movie_id)



In [185]:
findmovie('jumanji ')  #FROM JUMANJI TO jumanji 1995	TO index to the recommendation

,Users Like Us,Average Person,score,Title,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
2,1.000000,0.017544,57.000000,Jumanji (1995),2,jumanji 1995,adventure children fantasy,2 jumanji 1995 adventure children fantasy
157,0.104827,0.005603,18.708383,Casper (1995),158,casper 1995,adventure children,158 casper 1995 adventure children
314,0.140271,0.009420,14.890449,"Santa Clause, The (1994)",317,santa clause the 1994,comedy drama fantasy,317 santa clause the 1994 comedy drama fantasy
579,0.190799,0.020435,9.337002,Home Alone (1990),586,home alone 1990,children comedy,586 home alone 1990 children comedy
496,0.265460,0.030503,8.702668,Mrs. Doubtfire (1993),500,mrs doubtfire 1993,comedy drama,500 mrs doubtfire 1993 comedy drama
363,0.194193,0.022380,8.677210,"Mask, The (1994)",367,mask the 1994,action comedy crime fantasy,367 mask the 1994 action comedy crime fantasy
2527,0.119910,0.015103,7.939553,"Mummy, The (1999)",2617,mummy the 1999,action adventure comedy fantasy horror thriller,2617 mummy the 1999 action adventure comedy...
722,0.164027,0.021811,7.520473,Twister (1996),736,twister 1996,action adventure romance thriller,736 twister 1996 action adventure romance th...
580,0.190045,0.029915,6.352941,Ghost (1990),587,ghost 1990,comedy drama fantasy romance thriller,587 ghost 1990 comedy drama fantasy romance ...
313,0.171192,0.027427,6.241679,Stargate (1994),316,stargate 1994,action adventure sci fi,316 stargate 1994 action adventure sci fi


In [186]:
findmovie('jumanji 2017  ')

,Users Like Us,Average Person,score,Title,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
49688,1.000000,0.002132,469.025078,Jumanji: Welcome to the Jungle (2017),179401,jumanji welcome to the jungle 2017,action adventure children,179401 jumanji welcome to the jungle 2017 a...
45676,0.100313,0.000976,102.800017,The Fate of the Furious (2017),170875,the fate of the furious 2017,action crime drama thriller,170875 the fate of the furious 2017 action c...
47953,0.166144,0.001624,102.297651,The Hitman's Bodyguard (2017),175661,the hitman s bodyguard 2017,action comedy,175661 the hitman s bodyguard 2017 action co...
53868,0.203762,0.002039,99.956164,Ant-Man and the Wasp (2018),188301,ant man and the wasp 2018,action adventure comedy fantasy sci fi,188301 ant man and the wasp 2018 action adve...
51657,0.141066,0.001617,87.215407,Game Night (2018),183611,game night 2018,action comedy crime horror,183611 game night 2018 action comedy crime h...
48169,0.175549,0.002145,81.823690,Kingsman: The Golden Circle (2017),176101,kingsman the golden circle 2017,action adventure comedy,176101 kingsman the golden circle 2017 acti...
50419,0.147335,0.001818,81.044775,The Greatest Showman (2017),180985,the greatest showman 2017,drama,180985 the greatest showman 2017 drama
53525,0.115987,0.001470,78.881490,Solo: A Star Wars Story (2018),187595,solo a star wars story 2018,action adventure children sci fi,187595 solo a star wars story 2018 action a...
25060,0.144201,0.001985,72.643615,Pirates of the Caribbean: Dead Men Tell No Tal...,122896,pirates of the caribbean dead men tell no tal...,no genres listed,122896 pirates of the caribbean dead men tell...
32457,0.238245,0.003890,61.247261,Ready Player One,140956,ready player one,action sci fi thriller,140956 ready player one action sci fi thriller


In [187]:
find_similar_movies(179401)

,Users Like Us,Average Person,score,Title,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
49688,1.000000,0.002132,469.025078,Jumanji: Welcome to the Jungle (2017),179401,jumanji welcome to the jungle 2017,action adventure children,179401 jumanji welcome to the jungle 2017 a...
45676,0.100313,0.000976,102.800017,The Fate of the Furious (2017),170875,the fate of the furious 2017,action crime drama thriller,170875 the fate of the furious 2017 action c...
47953,0.166144,0.001624,102.297651,The Hitman's Bodyguard (2017),175661,the hitman s bodyguard 2017,action comedy,175661 the hitman s bodyguard 2017 action co...
53868,0.203762,0.002039,99.956164,Ant-Man and the Wasp (2018),188301,ant man and the wasp 2018,action adventure comedy fantasy sci fi,188301 ant man and the wasp 2018 action adve...
51657,0.141066,0.001617,87.215407,Game Night (2018),183611,game night 2018,action comedy crime horror,183611 game night 2018 action comedy crime h...
48169,0.175549,0.002145,81.823690,Kingsman: The Golden Circle (2017),176101,kingsman the golden circle 2017,action adventure comedy,176101 kingsman the golden circle 2017 acti...
50419,0.147335,0.001818,81.044775,The Greatest Showman (2017),180985,the greatest showman 2017,drama,180985 the greatest showman 2017 drama
53525,0.115987,0.001470,78.881490,Solo: A Star Wars Story (2018),187595,solo a star wars story 2018,action adventure children sci fi,187595 solo a star wars story 2018 action a...
25060,0.144201,0.001985,72.643615,Pirates of the Caribbean: Dead Men Tell No Tal...,122896,pirates of the caribbean dead men tell no tal...,no genres listed,122896 pirates of the caribbean dead men tell...
32457,0.238245,0.003890,61.247261,Ready Player One,140956,ready player one,action sci fi thriller,140956 ready player one action sci fi thriller


In [188]:
find_similar_movies(187593)



,Users Like Us,Average Person,score,Title,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
53524,1.000000,0.008639,115.752521,Deadpool 2 (2018),187593,deadpool 2 2018,action comedy sci fi,187593 deadpool 2 2018 action comedy sci fi
53868,0.121800,0.002044,59.584085,Ant-Man and the Wasp (2018),188301,ant man and the wasp 2018,action adventure comedy fantasy sci fi,188301 ant man and the wasp 2018 action adve...
32457,0.157486,0.003901,40.374161,Ready Player One,140956,ready player one,action sci fi thriller,140956 ready player one action sci fi thriller
53498,0.183088,0.004718,38.803402,Incredibles 2 (2018),187541,incredibles 2 2018,action adventure animation children,187541 incredibles 2 2018 action adventure a...
25075,0.292475,0.007989,36.609648,Untitled Spider-Man Reboot (2017),122926,untitled spider man reboot 2017,action adventure fantasy,122926 untitled spider man reboot 2017 actio...
25065,0.226532,0.006253,36.226941,Black Panther (2017),122906,black panther 2017,action adventure sci fi,122906 black panther 2017 action adventure s...
25068,0.566330,0.016936,33.438599,Avengers: Infinity War - Part I (2018),122912,avengers infinity war part i 2018,action adventure sci fi,122912 avengers infinity war part i 2018 ...
44449,0.186967,0.005717,32.703819,John Wick: Chapter Two (2017),168248,john wick chapter two 2017,action crime thriller,168248 john wick chapter two 2017 action cr...
25070,0.463926,0.014242,32.574121,Thor: Ragnarok (2017),122916,thor ragnarok 2017,action adventure sci fi,122916 thor ragnarok 2017 action adventure ...
25069,0.240497,0.007875,30.538963,Avengers: Infinity War - Part II (2019),122914,avengers infinity war part ii 2019,action adventure sci fi,122914 avengers infinity war part ii 2019 ...


In [189]:
findmovie('Deadpool 2 (2018)	')

,Users Like Us,Average Person,score,Title,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
53524,1.000000,0.008639,115.752521,Deadpool 2 (2018),187593,deadpool 2 2018,action comedy sci fi,187593 deadpool 2 2018 action comedy sci fi
53868,0.121800,0.002044,59.584085,Ant-Man and the Wasp (2018),188301,ant man and the wasp 2018,action adventure comedy fantasy sci fi,188301 ant man and the wasp 2018 action adve...
32457,0.157486,0.003901,40.374161,Ready Player One,140956,ready player one,action sci fi thriller,140956 ready player one action sci fi thriller
53498,0.183088,0.004718,38.803402,Incredibles 2 (2018),187541,incredibles 2 2018,action adventure animation children,187541 incredibles 2 2018 action adventure a...
25075,0.292475,0.007989,36.609648,Untitled Spider-Man Reboot (2017),122926,untitled spider man reboot 2017,action adventure fantasy,122926 untitled spider man reboot 2017 actio...
25065,0.226532,0.006253,36.226941,Black Panther (2017),122906,black panther 2017,action adventure sci fi,122906 black panther 2017 action adventure s...
25068,0.566330,0.016936,33.438599,Avengers: Infinity War - Part I (2018),122912,avengers infinity war part i 2018,action adventure sci fi,122912 avengers infinity war part i 2018 ...
44449,0.186967,0.005717,32.703819,John Wick: Chapter Two (2017),168248,john wick chapter two 2017,action crime thriller,168248 john wick chapter two 2017 action cr...
25070,0.463926,0.014242,32.574121,Thor: Ragnarok (2017),122916,thor ragnarok 2017,action adventure sci fi,122916 thor ragnarok 2017 action adventure ...
25069,0.240497,0.007875,30.538963,Avengers: Infinity War - Part II (2019),122914,avengers infinity war part ii 2019,action adventure sci fi,122914 avengers infinity war part ii 2019 ...


In [190]:
matching_movies = moviescsv[moviescsv['Title_cleaned'].str.contains("Avengers", case=False)]
matching_movies

,movieId,Title,Genres,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
2064,2153,"Avengers, The (1998)",Action|Adventure,2153,avengers the 1998,action adventure,2153 avengers the 1998 action adventure
17068,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,89745,avengers the 2012,action adventure sci fi imax,89745 avengers the 2012 action adventure sc...
22600,115727,Crippled Avengers (Can que) (Return of the 5 D...,Action|Adventure,115727,crippled avengers can que return of the 5 d...,action adventure,115727 crippled avengers can que return of ...
25059,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,122892,avengers age of ultron 2015,action adventure sci fi,122892 avengers age of ultron 2015 action a...
25068,122912,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,122912,avengers infinity war part i 2018,action adventure sci fi,122912 avengers infinity war part i 2018 ...
25069,122914,Avengers: Infinity War - Part II (2019),Action|Adventure|Sci-Fi,122914,avengers infinity war part ii 2019,action adventure sci fi,122914 avengers infinity war part ii 2019 ...
30334,135979,Next Avengers: Heroes of Tomorrow (2008),Action|Animation|Children|Sci-Fi,135979,next avengers heroes of tomorrow 2008,action animation children sci fi,135979 next avengers heroes of tomorrow 2008...
30432,136257,Avengers Grimm (2015),Action|Adventure|Fantasy,136257,avengers grimm 2015,action adventure fantasy,136257 avengers grimm 2015 action adventure ...
34537,145676,3 Avengers (1964),(no genres listed),145676,3 avengers 1964,no genres listed,145676 3 avengers 1964 no genres listed
35220,147238,The New Adventures of the Elusive Avengers (1968),Action|Adventure|Children,147238,the new adventures of the elusive avengers 1968,action adventure children,147238 the new adventures of the elusive aveng...


In [191]:
findmovie('Avengers: Infinity War	')

,Users Like Us,Average Person,score,Title,movieId_cleaned,Title_cleaned,Genres_cleaned,all_data
25068,1.000000,0.017032,58.712307,Avengers: Infinity War - Part I (2018),122912,avengers infinity war part i 2018,action adventure sci fi,122912 avengers infinity war part i 2018 ...
25069,0.296399,0.007920,37.425973,Avengers: Infinity War - Part II (2019),122914,avengers infinity war part ii 2019,action adventure sci fi,122914 avengers infinity war part ii 2019 ...
25070,0.486347,0.014323,33.956435,Thor: Ragnarok (2017),122916,thor ragnarok 2017,action adventure sci fi,122916 thor ragnarok 2017 action adventure ...
25075,0.271468,0.008034,33.789130,Untitled Spider-Man Reboot (2017),122926,untitled spider man reboot 2017,action adventure fantasy,122926 untitled spider man reboot 2017 actio...
25065,0.210131,0.006289,33.415043,Black Panther (2017),122906,black panther 2017,action adventure sci fi,122906 black panther 2017 action adventure s...
53524,0.288880,0.008688,33.250570,Deadpool 2 (2018),187593,deadpool 2 2018,action comedy sci fi,187593 deadpool 2 2018 action comedy sci fi
32457,0.115156,0.003923,29.356154,Ready Player One,140956,ready player one,action sci fi thriller,140956 ready player one action sci fi thriller
53498,0.134547,0.004745,28.355376,Incredibles 2 (2018),187541,incredibles 2 2018,action adventure animation children,187541 incredibles 2 2018 action adventure a...
25071,0.388207,0.015442,25.140451,Guardians of the Galaxy 2 (2017),122918,guardians of the galaxy 2 2017,action adventure sci fi,122918 guardians of the galaxy 2 2017 action...
52321,0.102493,0.004145,24.725996,A Quiet Place (2018),185029,a quiet place 2018,drama horror thriller,185029 a quiet place 2018 drama horror thriller
